<a href="https://colab.research.google.com/github/Athar47/Emotion-Analysis-Via-Text/blob/main/Model_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import re
import nltk
import string
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.pipeline import Pipeline

nltk.download('wordnet')
nltk.download("stopwords")
stop_words = set(stopwords.words("english"))



In [ ]:
# Read datasets
df_train = pd.read_csv('train.txt', names=['Text', 'Emotion'], sep=';')
df_val = pd.read_csv('val.txt', names=['Text', 'Emotion'], sep=';')
df_test = pd.read_csv('test.txt', names=['Text', 'Emotion'], sep=';')

In [ ]:
# Removing duplicated values in df_train
df_train = df_train.drop_duplicates().reset_index(drop=True)

# Removing duplicated text in df_val
df_val = df_val.drop_duplicates(subset=['Text']).reset_index(drop=True)


In [ ]:
def lemmatization(text):
    # Lemmatize each word in the text
    lemmatized_text = [lemmatizer.lemmatize(word) for word in text.split()]
    return " ".join(lemmatized_text)

def remove_stop_words(text):
    # Remove stop words from the text
    filtered_text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(filtered_text)

def removing_numbers(text):
    # Remove digits from the text
    return ''.join([char for char in text if not char.isdigit()])

def lower_case(text):
    # Convert text to lowercase
    return " ".join([word.lower() for word in text.split()])

def removing_punctuations(text):
    # Remove punctuations, extra whitespace, and specific characters
    text = re.sub('[%s]' % re.escape("""!"#$%&'()*+,،-./:;<=>؟?@[\]^_`{|}~"""), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\s+', ' ', text)
    return text.strip()

def removing_urls(text):
    # Remove URLs from the text
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def remove_small_sentences(df):
    # Remove rows with small sentences (less than 3 words)
    df['Text'] = df['Text'].apply(lambda text: np.nan if len(str(text).split()) < 3 else text)

def normalize_text(df):
    # Apply various text normalization functions to the 'Text' column
    df['Text'] = df['Text'].apply(lower_case)
    df['Text'] = df['Text'].apply(remove_stop_words)
    df['Text'] = df['Text'].apply(removing_numbers)
    df['Text'] = df['Text'].apply(removing_punctuations)
    df['Text'] = df['Text'].apply(removing_urls)
    df['Text'] = df['Text'].apply(lemmatization)
    return df

def normalized_sentence(sentence):
    # Apply various text normalization functions to a single sentence
    sentence = lower_case(sentence)
    sentence = remove_stop_words(sentence)
    sentence = removing_numbers(sentence)
    sentence = removing_punctuations(sentence)
    sentence = removing_urls(sentence)
    sentence = lemmatization(sentence)
    return sentence


In [ ]:
# Normalize the text for training, testing, and validation datasets
df_train = normalize_text(df_train)
df_test = normalize_text(df_test)
df_val = normalize_text(df_val)

In [ ]:
# Extract features and labels for training data
X_train = df_train['Text'].values
y_train = df_train['Emotion'].values

# Extract features and labels for test data
X_test = df_test['Text'].values
y_test = df_test['Emotion'].values

# Extract features and labels for validation data
X_val = df_val['Text'].values
y_val = df_val['Emotion'].values

In [ ]:
def train_model(model, data, targets):
    # Create a Pipeline with TfidfVectorizer and the given model
    text_clf = Pipeline([('vect', TfidfVectorizer()),
                         ('clf', model)])
    # Fit the model on the data and targets
    text_clf.fit(data, targets)
    return text_clf

In [ ]:
# Train the model with the training data
RF = train_model(RandomForestClassifier(random_state=0), X_train, y_train)

# Test the model with the test data
y_pred = RF.predict(X_test)

# Calculate the accuracy
RF_accuracy = accuracy_score(y_test, y_pred)
print('Accuracy: ', RF_accuracy, '\n')

In [ ]:
##Classification Report
print(classification_report(y_test, y_pred))

In [ ]:
# User input prediction
user_input = input("Enter a text (at least 5 words): ")
while len(user_input.split()) < 5:
    print("Please enter at least 5 words.")
    user_input = input("Enter a text (at least 5 words): ")

# Normalize the user input
normalized_input = normalize_text(pd.DataFrame({'Text': [user_input]})).iloc[0]['Text']

# Make prediction using the trained RandomForestClassifier model
user_pred = RF.predict([normalized_input])

# Print the predicted emotion
print("Predicted Emotion:", user_pred[0])